In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
import sys

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

In [78]:
fmin = 5.725e9
fMAX = 5.875e9
f0 = 5.8e9
B = fMAX - fmin
T = 600e-6
alpha = 2 * (fMAX - fmin) / T
c = 3e8

In [79]:
t = np.arange( 0, T, 100e-9)

In [80]:
def sr( t, r, n ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    idx = np.where( t <= T/2  )
    y[0:3000] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * (t[0:3000] + n*T) * tau )
    
    # down ramp
    idx = np.where( T/2 < t )
    y[3000:6000] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * (t[3000:6000] + n*T) * tau )
    
    return np.real( y )

Let us build the equivalent of 10 files of data

In [43]:
rampsPerFile = 1500
samplesPerRamp = 6000
nbFiles = 20
adc = np.zeros( nbFiles * rampsPerFile * samplesPerRamp )

In [44]:
Laz = V * nbFiles * rampsPerFile * T
phi = np.arctan( (Laz/2) / rg_corner )
print( "Laz = {:.0f}, phi = {}".format( Laz, phi * 180 / np.pi ) )

Laz = 720, phi = 56.26408665659793


In [45]:
a = 0.603354564402
b = a * 180 / (30 * np.pi) # see the Newton method at the bottom of the notebook
def wa( az, rg0 ):
    Rn = ( rg0**2 + az**2 )**0.5
    wa = np.sinc( b * np.arccos( rg0 / Rn ) )**2
    return wa

In [87]:
V = 40
hPlane = 430
hCorner = 90
rg_corner = ( hPlane - hCorner ) * 2**0.5
gr_corner = hPlane - hCorner
az_corner = 186
noise = 1

for n in range( nbFiles * rampsPerFile):
    az = n * V * T
    Rn = ( rg_corner**2 + (az - az_corner)**2 )**0.5
    # azmimuth beam pattern effect + 
    adc[ n * samplesPerRamp : (n+1) * samplesPerRamp ] = sr( t, Rn, n ) * wa( az - az_corner, rg_corner ) \
    + np.random.randn( 6000 ) * noise
    if n%1000 == 0:
        phi = np.arccos( rg_corner / Rn ) * 180 / np.pi
        print( "{}, wa = {:.1f}, phi = {:.1f}".format( n, wa( az - az_corner, rg_corner ), phi ) )

0, wa = 0.5, phi = 21.1
1000, wa = 0.6, phi = 18.6
2000, wa = 0.7, phi = 16.0
3000, wa = 0.8, phi = 13.3
4000, wa = 0.9, phi = 10.6
5000, wa = 0.9, phi = 7.8
6000, wa = 1.0, phi = 5.0
7000, wa = 1.0, phi = 2.1
8000, wa = 1.0, phi = 0.7
9000, wa = 1.0, phi = 3.6
10000, wa = 0.9, phi = 6.4
11000, wa = 0.9, phi = 9.2
12000, wa = 0.8, phi = 12.0
13000, wa = 0.7, phi = 14.7
14000, wa = 0.7, phi = 17.3
15000, wa = 0.6, phi = 19.9
16000, wa = 0.5, phi = 22.4
17000, wa = 0.4, phi = 24.8
18000, wa = 0.3, phi = 27.1
19000, wa = 0.3, phi = 29.3
20000, wa = 0.2, phi = 31.4
21000, wa = 0.2, phi = 33.5
22000, wa = 0.1, phi = 35.4
23000, wa = 0.1, phi = 37.3
24000, wa = 0.1, phi = 39.0
25000, wa = 0.0, phi = 40.7
26000, wa = 0.0, phi = 42.3
27000, wa = 0.0, phi = 43.9
28000, wa = 0.0, phi = 45.3
29000, wa = 0.0, phi = 46.7


In [88]:
adc_reshaped = adc.reshape( (nbFiles * rampsPerFile, samplesPerRamp) )

In [89]:
RD = build_rd_from_data_sim( adc_reshaped, samplesPerRamp )

shape of the samples matrix = (30000, 6000)
Data are NOT shifted


In [90]:
ifft_RD = np.fft.ifft( RD, axis=1 )

In [91]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( ifft_RD[::10,:] ) ) )

In [92]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,500] ) ))
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,501] ) ))

# Save simulated data

In [93]:
firstFile = 0
lastFile = nbFiles - 1
data_dir = "/home/pleroy/DATA/SIMU"

In [94]:
coupling_RD = np.average(RD, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}".format(firstFile, lastFile), coupling_RD )

In [95]:
np.save(data_dir + '/RD_files_{}_{}'.format(firstFile, lastFile), RD)

In [ ]:
V * rampsPerFile * nbFiles * T / 2

In [ ]:
340*2**0.5

### Newton's method
f(x) = sin(x) / x

f'(x) = cos(x) / x - sin(x) / x^2

x(n+1) = x(n) - f(x(n)) / f'(x(n))

x(n+1) = x(n) - [f(x(n)) - y0] / f'(x(n))

In [14]:
y0 = 0.5
def newton( xn ):
    f_xn = np.sinc( xn )
    #f_prime_xn = np.cos(xn) / xn - np.sin(xn) / xn**2
    f_prime_xn = np.cos(np.pi * xn) / xn - np.sin(np.pi*xn) / (np.pi*xn**2)
    xn_plus_1 = xn - ( f_xn - y0 ) / f_prime_xn
    return xn_plus_1

In [15]:
x = 1
for k in range( 10 ):
    x = newton( x )
    print( x )

0.5
0.607300918301
0.603356946177
0.603354564403
0.603354564402
0.603354564402
0.603354564402
0.603354564402
0.603354564402
0.603354564402


In [35]:
a = 0.603354564402
b = a * 180 / (30 * np.pi)

In [36]:
x = 30 * np.pi / 180
np.sinc( b * x )

0.49999999999947636